<a href="https://colab.research.google.com/github/Lasvut/DLIGroupAssignmentJ/blob/main/GroupJ_DLIAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [7]:
#Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks


#Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifierCV
from sklearn.feature_selection import SelectFromModel


#Reproductibility
SEED = 42



# Data Loading

In [2]:
#Mount Drive

from google.colab import drive
drive.mount('/content/drive')
data_path = ('/content/drive/MyDrive/PhiUSIIL_Phishing_URL_Dataset.csv')

#Import Dataset
df = pd.read_csv(data_path)

print('Shape: ', df.shape)
print('Columns: ', list(df.columns)[:10], '...')
df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape:  (235795, 56)
Columns:  ['FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength', 'IsDomainIP', 'TLD', 'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb'] ...


,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


# Pre-processing



In [5]:
text_col = ['FILENAME', 'URL', 'Domain', 'Title']
target_col = 'label'

print('Original shape: ', df.shape)

#==========================Data Cleaning===========================

#Drop empty rows
df = df.dropna(how='all')

#Drop rows with missing label; label: phishing 1, legit 0
df = df.dropna(subset=[target_col])

#Drop duplicaeds by FILENAME and URL
if 'FILENAME' in df.columns:
  df = df.drop_duplicates(subset='FILENAME')
elif 'URL' in df.columns:
  df = df.drop_duplicates(subset='URL')

#Strop whitespace
for col in df.select_dtypes(include=['object']).columns:
  df[col] = df[col].astype(str).str.strip()

#Drop raw text col
for c in text_col:
  if c in df.columns:
    df = df.drop(columns=[c])

#Ensure target col exist
assert target_col in df.columns, f'Target column "{target_col}" not found.'

#Convert columns to numeric
feature_col = [c for c in df.columns if c != target_col]
X_raw = df[feature_col].apply(pd.to_numeric, errors='coerce')
y = df[target_col].astype(int)

#===========================Noice filtering==========================

#Ratios and Prob should always between 0 and 1
ratio_col = [c for c in X_raw.columns if 'Ratio' in c or 'Prob' in c]
for col in ratio_col:
  if col in X_raw.columns:
    bad_idx = X_raw[(X_raw[col] < 0) | (X_raw[col] > 1)].index
    if len(bad_idx) > 0:
      print(f'Dropping {len(bad_idx)} noisy rows from {col}')
      X_raw = X_raw.drop(index=bad_idx)
      y = y.drop(index=bad_idx)

#Remove negative values from count columns
count_col = [c for c in X_raw.columns if 'Length' in c or 'NoOf' in c or 'Line' in c]
for col in count_col:
  if col in X_raw.columns:
    bad_idx = X_raw[X_raw[col] < 0].index
    if len(bad_idx) > 0:
      print(f'Dropping {len(bad_idx)} rows with negative value in {col}')
      X_raw = X_raw.drop(index=bad_idx)
      y = y.drop(index=bad_idx)

#Fills NaNs with medians
X_raw = X_raw.fillna(X_raw.median(numeric_only=True))

print('After cleaning: ', X_raw.shape)

#=============================Class balancing=========================================

#Class balancing with ROS
print('Initial Class Distribution: ')
print(y.value_counts(dropna=False).sort_index())

ros = RandomOverSampler(random_state=SEED)
X_bal, y_bal = ros.fit_resample(X_raw, y)

print('\nAfter RandomOverSampler: ')
print(y_bal.value_counts(dropna=False).sort_index())

#============================Train-test split (80/20)====================================

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_bal, y_bal, test_size=0.2, random_state=SEED, stratify=y_bal
)

print('\nTrain shape: ', X_train_raw.shape, 'Test shape: ', X_test_raw.shape)

Original shape:  (235795, 56)
After cleaning:  (235795, 51)
Initial Class Distribution: 
label
0    100945
1    134850
Name: count, dtype: int64

After RandomOverSampler: 
label
0    134850
1    134850
Name: count, dtype: int64

Train shape:  (215760, 51) Test shape:  (53940, 51)


Optional Pre-processing: Ridge Regression-based Feature Selection

In [10]:
def ridge_feature_selection(X_train_raw, X_test_raw, y_train, feature_col):
  #===========================Scalar features============================

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train_raw)
  X_test_scaled = scaler.fit_transform(X_test_raw)

  #==========================Ridge with Cross-validation==================

  ridge = RidgeClassifierCV(
      alphas = np.logspace(-3, 3, 13), cv = 5
  )

  #Select Informative Feature
  selector = SelectFromModel(estimator=ridge, threshold='median')
  selector.fit(X_train_scaled, y_train)

  #Check number of selected features
  selected_mask = selector.get_support()
  num_selected = selected_mask.sum()

  #========================Transform data================================

  X_train_fs = selector.transform(X_train_scaled)
  X_test_fs = selector.transform(X_test_scaled)

  #Track selected features
  selected_features = np.array(feature_col)[selected_mask]

  print(f'Selected features: {num_selected}/{len(feature_col)}')
  print('Top features: ', list(selected_features))

  return X_train_fs, X_test_fs, selected_features

Selected features: 26/51
Top features:  [np.str_('URLLength'), np.str_('IsDomainIP'), np.str_('URLSimilarityIndex'), np.str_('TLDLegitimateProb'), np.str_('TLDLength'), np.str_('NoOfSubDomain'), np.str_('NoOfLettersInURL'), np.str_('LetterRatioInURL'), np.str_('NoOfDegitsInURL'), np.str_('DegitRatioInURL'), np.str_('NoOfEqualsInURL'), np.str_('NoOfAmpersandInURL'), np.str_('NoOfOtherSpecialCharsInURL'), np.str_('SpacialCharRatioInURL'), np.str_('IsHTTPS'), np.str_('DomainTitleMatchScore'), np.str_('URLTitleMatchScore'), np.str_('HasFavicon'), np.str_('Robots'), np.str_('NoOfURLRedirect'), np.str_('HasDescription'), np.str_('HasSocialNet'), np.str_('HasSubmitButton'), np.str_('HasHiddenFields'), np.str_('HasCopyrightInfo'), np.str_('NoOfJS')]


# Model

ANN - Tan Mu Yuan(TP069241)

In [ ]:
def build_ann(input_dim: int) -> tf.keras.Model:
  inputs = layers.Input(shape=(input_dim,))

  #layer 1
  x = layers.Dense(128, actication='relu')(inputs)
  x = layers.Dropout(0.3)(x)

  #layer 2
  x = layers.Dense(64, actication='relu')(x)
  x = layers.Dropout(0.2)(x)

  #layer 3
  x = layers.Dense(32, actication='relu')(inputs)

  #Output layer
  outputs = layers.Dense(1, activation='sigmoid')(x)

  model = model.Model(inputs,outputs, name='Phishing URL Detection using ANN')
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss = 'binary_crossentropy',
      metrics=['accuracy']
  )
  return model

#Build mode using selected features
input_dim = X_train_fs.shape[1]
model = build_ann(input_dim)
model.summary()
print('Trainable Parameters: ', model.count_params())


# Training

ANN - Tan Mu Yuan(TP069241)

# Evaluation

# Save Model